In [ ]:
import os

import numpy as np
import math
import uproot as uproot
import pickle
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import ticker
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
from matplotlib import gridspec
import dunestyle.matplotlib as dunestyle

import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import median_abs_deviation
from scipy.interpolate import CubicSpline
from scipy.optimize import curve_fit
import scipy.linalg as la
import scipy.optimize as opt
from scipy.optimize import Bounds, LinearConstraint
from scipy.stats import chisquare

from landaupy import langauss

from branches import *
from pandas_helpers import *

In [ ]:
PDG = {
    "muon": [13, "muon", 0.105,],
    "proton": [2212, "proton", 0.938272,], 
    "neutron": [2112, "neutron", 0.9395654,], 
    "pizero": [111, "pizero", 0.1349768], 
    "piplus": [211, "piplus", 0.13957039], 
    "piminus": [-211, "piminus", 0.13957039], 
    "argon": [1000180400, "argon", (18*0.938272 + 22*0.9395654)], 
    "gamma": [22, "gamma", 0 ], 
    "lambda": [3122, "lambda", 1.115683], 
    "kaon_p": [321, "kaon_p",  0.493677], 
    "sigma_p": [3222, "sigma_p", 1.18936], 
    "kaon_0": [311, "kaon_0", 0.497648], 
    "sigma_0": [3212, "sigma_0", 1.19246], 
    "lambda_p_c": [4122, "lambda_p_c", 2.28646], 
    "sigma_pp_c": [4222, "sigma_pp_c", 2.45397], 
    "electron": [11, "electron", 0.510998950], 
    "sigma_p_c": [4212, "sigma_p_c", 2.4529],
}

THRESHOLD = {"muon": 0.175, "proton": 0.3, "proton_stub": 0.2, "picharged": 0.07, "pizero":0}

In [ ]:
def InFV(data): # cm
    xmin = -199.15 + 10
    ymin = -200. + 10
    zmin = 0.0 + 10
    xmax = 199.15 - 10
    ymax =  200. - 10
    zmax =  500. - 50
    return (data.x > xmin) & (data.x < xmax) & (data.y > ymin) & (data.y < ymax) & (data.z > zmin) & (data.z < zmax)

def InBeam(t):
    return (t > 0.) & (t < 1.800)

def Avg(df, pid, drop_0=True):  # average score of 3 planes, exclude value if 0
    if drop_0:
        df = df.replace(0, np.nan)
    average = df[[("chi2pid", "I0", "chi2_"+pid), ("chi2pid", "I1", "chi2_"+pid), ("chi2pid", "I2", "chi2_"+pid)]].mean(skipna=drop_0, axis=1)
    return average

def Signal(df): # signal definition
    is_fv = InFV(df.nu.position)
    is_numu = (df.nu.pdg == 14)
    is_cc = (df.nu.iscc == 1)
    is_1p0pi = (df.mult_proton_def == 1) & (df.mult_picharged_def == 0) & (df.mult_pizero == 0)
    return is_fv & is_numu & is_cc # & is_1p0pi

In [ ]:
fname = '/Users/sungbino/Study/FNAL/SBND/data/flatcaf/2024A_test/2024_genie_cv/reco2_hadd.root'
#fname = '/Users/sungbino/Study/FNAL/SBND/data/flatcaf/2024A_test/2024_genie_cv/reco2_5_files_hadd.root'
events = uproot.open(fname+":recTree")

In [ ]:
# MC truth
nudf = loadbranches(events, mc_branches)
nudf = nudf.rec.mc

nuprimdf = loadbranches(events, mc_prim_branches)
nuprimdf = nuprimdf.rec.mc.nu
nuprimdf[("prim","totp","")] = np.sqrt((nuprimdf.prim.startp.x)**2+(nuprimdf.prim.startp.y)**2+(nuprimdf.prim.startp.z)**2) # |momentum| branch

# primary track multiplicity
mult_muon = (nuprimdf.prim.pdg == 13).groupby(level=[0,1]).sum()
mult_muon_def = ((nuprimdf.prim.pdg == 13) & (nuprimdf.prim.totp > THRESHOLD["muon"])).groupby(level=[0,1]).sum()
mult_proton = (nuprimdf.prim.pdg == 2212).groupby(level=[0,1]).sum()
mult_proton_def = ((nuprimdf.prim.pdg == 2212) & (nuprimdf.prim.totp > THRESHOLD["proton"])).groupby(level=[0,1]).sum()
mult_proton_stub_def = ((nuprimdf.prim.pdg == 2212) & (nuprimdf.prim.totp > THRESHOLD["proton_stub"])).groupby(level=[0,1]).sum()
mult_picharged = (np.abs(nuprimdf.prim.pdg) == 211).groupby(level=[0,1]).sum()
mult_picharged_def = ((np.abs(nuprimdf.prim.pdg) == 211) & (nuprimdf.prim.totp > THRESHOLD["picharged"])).groupby(level=[0,1]).sum()
mult_pizero = (np.abs(nuprimdf.prim.pdg) == 211).groupby(level=[0,1]).sum()
mult_pizero_def = ((np.abs(nuprimdf.prim.pdg) == 111) & (nuprimdf.prim.totp > THRESHOLD["pizero"])).groupby(level=[0,1]).sum()
mult_electron = (nuprimdf.prim.pdg == 11).groupby(level=[0,1]).sum()
mult_photon = (nuprimdf.prim.pdg == 22).groupby(level=[0,1]).sum()
nudf['mult_muon'] = mult_muon
nudf['mult_muon_def'] = mult_muon_def
nudf['mult_proton'] = mult_proton
nudf['mult_proton_def'] = mult_proton_def
nudf['mult_proton_stub_def'] = mult_proton_stub_def
nudf['mult_picharged'] = mult_picharged
nudf['mult_picharged_def'] = mult_picharged_def
nudf['mult_pizero'] = mult_pizero
nudf['mult_pizero_def'] = mult_pizero_def
nudf['mult_electron'] = mult_electron
nudf['mult_photon'] = mult_photon

# truth match
slcdf = loadbranches(events, slc_branches)
slcdf = slcdf.rec

slcdf.loc[np.invert(slcdf[("slc","tmatch","eff")] > 0.5) & (slcdf[("slc","tmatch","idx")] >= 0), ("slc","tmatch","idx")] = np.nan
slcdf["tmatch_index"] = slcdf[("slc", "tmatch", "idx")]

matchdf = pd.merge(slcdf.reset_index(), 
                 nudf.reset_index(),
                 left_on=[("entry", "",""), ("slc","tmatch", "idx")], # entry index -> neutrino index
                 right_on=[("entry", "",""), ("rec.mc.nu..index", "","")], 
                 how="left", # Keep every slc
                 )

matchdf = matchdf.set_index(["entry", "rec.slc..index"], verify_integrity=True)

In [ ]:
# reco pfps
pfptrkdf = loadbranches(events, pfp_trk_branches)
pfptrkdf = pfptrkdf.rec.slc.reco.pfp

pfptrkchi2df = loadbranches(events, pfp_trk_chi2_branches)
pfptrkchi2df = pfptrkchi2df.rec.slc.reco.pfp.trk

pfptrkdf = pfptrkdf.join(pfptrkchi2df)

pfptruthdf = loadbranches(events, pfp_trk_mc_branches)
pfptruthdf = pfptruthdf.rec.slc.reco.pfp.trk.truth

pfpdf = pd.merge(pfptrkdf, pfptruthdf, left_index=True, right_index=True, how="inner")

pandoradf = loadbranches(events, pandora_branches)
pandoradf = pandoradf.rec.slc

pfpdf = pd.merge(pfpdf, pandoradf, left_index=True, right_index=True, how="inner")

# merge all
masterdf = pd.merge(matchdf, pfpdf, left_index=True, right_index=True, how="inner")

In [ ]:
## reco pfp track hits
pfptrkhit0df = loadbranches(events, pfp_trk_hit0_branches)
pfptrkhit1df = loadbranches(events, pfp_trk_hit1_branches)
pfptrkhit2df = loadbranches(events, pfp_trk_hit2_branches)
pfptrkhit0df = pfptrkhit0df.rec.slc.reco.pfp.trk.calo.I0.points
pfptrkhit1df = pfptrkhit1df.rec.slc.reco.pfp.trk.calo.I1.points
pfptrkhit2df = pfptrkhit2df.rec.slc.reco.pfp.trk.calo.I2.points

pfptrkhit0df.index.set_names("rec.slc.reco.pfp.trk.calo.points..index", level="rec.slc.reco.pfp.trk.calo.0.points..index", inplace=True)
pfptrkhit1df.index.set_names("rec.slc.reco.pfp.trk.calo.points..index", level="rec.slc.reco.pfp.trk.calo.1.points..index", inplace=True)
pfptrkhit2df.index.set_names("rec.slc.reco.pfp.trk.calo.points..index", level="rec.slc.reco.pfp.trk.calo.2.points..index", inplace=True)

In [ ]:
# reco vertex in FV
cut = InFV(masterdf.slc.vertex)
df_vertexcut = masterdf[cut]

In [ ]:
# cosmic rejection
cut_nuscore = (df_vertexcut.slc.nu_score.squeeze() > 0.4)
cut_fmatch = ((df_vertexcut.slc.fmatch.score < 7.0) & (InBeam(df_vertexcut.slc.fmatch.time)))
df_cosmiccut = df_vertexcut[cut_nuscore & cut_fmatch]

In [ ]:
# Long muon selection
MUCUT_LEN_THRES = 100
cut_len = (df_cosmiccut.trk.len > MUCUT_LEN_THRES)
primary_muons = df_cosmiccut[cut_len]
primary_muons = primary_muons.sort_values(["entry", "rec.slc..index", ("trk","len","")], ascending=[True, True, False])
leading_muon = primary_muons.groupby(["entry", "rec.slc..index"]).head(1) 
slc_idx_1mu = leading_muon.reset_index().set_index(["entry", "rec.slc..index"]).index.unique()
df_1mucut = df_cosmiccut.reset_index().set_index(["entry", "rec.slc..index"]).loc[slc_idx_1mu]
df_1mucut = df_1mucut.reset_index().set_index(["entry", "rec.slc..index", "rec.slc.reco.pfp..index"], verify_integrity=True)

In [ ]:
# Track score cut
cut_trk_score = (leading_muon[('reco', 'pfp', 'trackScore')] > 0.5)
trk_muon = leading_muon[cut_trk_score]

In [ ]:
# Contained muon selection
#cut_contained = (InFV(df_1mucut[('trk', 'start')])) & InFV(df_1mucut[('trk', 'end')])
cut_contained = (InFV(trk_muon[('trk', 'start')])) & InFV(trk_muon[('trk', 'end')])
df_contained_muon = trk_muon[cut_contained]

In [ ]:
hit_branches = pfptrkhit0df.columns.tolist()
hit0_sel_df = pd.merge(pfptrkhit0df.reset_index(), df_contained_muon.reset_index(),
                        left_on=[("entry"), ("rec.slc..index"), ("rec.slc.reco.pfp..index")],
                        right_on=[("entry"), ("rec.slc..index"), ("rec.slc.reco.pfp..index")], 
                        how="inner",
                        )

hit1_sel_df = pd.merge(pfptrkhit1df.reset_index(), df_contained_muon.reset_index(),
                        left_on=[("entry"), ("rec.slc..index"), ("rec.slc.reco.pfp..index")],
                        right_on=[("entry"), ("rec.slc..index"), ("rec.slc.reco.pfp..index")], 
                        how="inner",
                        )

hit2_sel_df = pd.merge(pfptrkhit2df.reset_index(), df_contained_muon.reset_index(),
                        left_on=[("entry"), ("rec.slc..index"), ("rec.slc.reco.pfp..index")],
                        right_on=[("entry"), ("rec.slc..index"), ("rec.slc.reco.pfp..index")], 
                        how="inner",
                        )

In [ ]:
pdg_list = hit0_sel_df[('p', 'pdg', '')].value_counts()
print(pdg_list)

In [ ]:
## Make pickle file
out_path = "/Users/sungbino/Study/FNAL/SBND/calib/chi2_pid/pickle/"

with open(os.path.join(out_path, "hit0_sel_df.pkl"), "wb") as f:
    pickle.dump(hit0_sel_df, f)